# TOPIC NAME

## Introduction

This guide will show you how to:

* TODO
* TODO

By the end of it, you will TODO!

## Setup

Install dependencies

In [ ]:
pip install neptune-client TODO

## Step 1: TODO

TODO

In [ ]:
# imports
import neptune
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# set project
neptune.init(api_token='ANONYMOUS',
             project_qualified_name='shared/pytorch-integration')

PARAMS = {'DEVICE': torch.device("cuda" if torch.cuda.is_available() else "cpu"),
          'ITERATIONS': 1000,
          'batch_size': 64,
          'lr': 0.005,
          'momentum': 0.9}

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# start experiment
neptune.create_experiment(name='pytorch-example', params=PARAMS)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))])),
   batch_size=PARAMS['batch_size'],
   shuffle=True)

model = Net().to(PARAMS['DEVICE'])

optimizer = optim.SGD(model.parameters(), PARAMS['lr'], PARAMS['momentum'])

for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(PARAMS['DEVICE']), target.to(PARAMS['DEVICE'])
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)

    # log loss to neptune
    neptune.log_metric('batch_loss', batch_idx, loss.data.cpu().numpy())

    loss.backward()
    optimizer.step()
    if batch_idx == PARAMS['ITERATIONS']:
        break

## Step 2: Initialize Neptune

Connects your script to Neptune application. 

In [ ]:
import neptune

neptune.init(
    api_token="ANONYMOUS",
    project_qualified_name="shared/onboarding"
)

You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune project `project_qualified_name`.

---

**Note:** 


Instead of logging data to the public project 'shared/onboarding' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your Neptune API token

![image](https://neptune.ai/wp-content/uploads/get_token.gif)

2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your username to the ``project_qualified_name`` argument of the ``neptune.init()`` method: ``project_qualified_name='YOUR_USERNAME/sandbox``. Keep `/sandbox` at the end, the `sandbox` project that was automatically created for you.

For example:

```python
neptune.init(project_qualified_name='funky_steve/sandbox', 
             api_token='eyJhcGlfYW908fsdf23f940jiri0bn3085gh03riv03irn',
            )
```

---

## Step 3: Create an experiment

In [ ]:
neptune.create_experiment(name='great-idea')

This opens a new "experiment" namespace in Neptune to which you can log various objects.

Click on the link above to open this experiment in Neptune.

For now it is empty but keep the tab with experiment open to see what happens next. 

## Step 4: TODO

TODO

In [ ]:
TODO

---

**Note:**
   
When you track experiments with Neptune in Jupyter notebooks you need to explicitly stop the experiment by running `neptune.stop()`.

If you are running Neptune in regular `.py` scripts it will stop automatically when your code stops running.

---

In [ ]:
neptune.stop()